In [1]:
import pandas as pd
import re
import datetime
from pathlib import Path

tweets_csv = Path('data/tweets_elonmusk_raw_FINAL.csv')
tweets_df = pd.read_csv(tweets_csv)

In [2]:
#check Tweet counts with thousand differentiator (.) and (K)
#WHERE only K - REPLACE with 3 zeros (000)
#WHERE dot (.) and K - REMOVE dot (.) and REPLACE K with 2 zeros (00)
#REPLACE M (million) with 6 zeros (000000)

######
######NOTE: THIS NEEDS TO BE REVISIT. IT APPEARS IT WILL REPLACE ALL K's WITH 00 FROM OTHER COLUMNS######
######
tweets_df.loc[(tweets_df.tweet_like_count.str.match('.*[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_like_count.str.match('.*[K,k]') == True)].replace('[K,k]','00', regex = True).replace('[.]','', regex = True)
tweets_df.loc[(tweets_df.tweet_like_count.str.match('[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_like_count.str.match('[K,k]') == True)].replace('[K,k]','000', regex = True)
tweets_df['tweet_like_count'] = tweets_df['tweet_like_count'].str.replace(r'M', '000000')

tweets_df.loc[(tweets_df.tweet_retweet_count.str.match('.*[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_retweet_count.str.match('.*[K,k]') == True)].replace('[K,k]','00', regex = True).replace('[.]','', regex = True)
tweets_df.loc[(tweets_df.tweet_retweet_count.str.match('[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_retweet_count.str.match('[K,k]') == True)].replace('[K,k]','000', regex = True)

tweets_df.loc[(tweets_df.tweet_reply_count.str.match('.*[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_reply_count.str.match('.*[K,k]') == True)].replace('[K,k]','00', regex = True).replace('[.]','', regex = True)
tweets_df.loc[(tweets_df.tweet_reply_count.str.match('[K,k]') == True)] = tweets_df.loc[(tweets_df.tweet_reply_count.str.match('[K,k]') == True)].replace('[K,k]','000', regex = True)
######
######END
######


#Tweets without content such as images will be filled with 'no content'
#reply, retweets and likes will be populated with 0 if not available
tweets_df['tweet_content'] = tweets_df['tweet_content'].fillna('NO CONTENT')
tweets_df['tweet_reply_count'] = tweets_df['tweet_reply_count'].fillna('0')
tweets_df['tweet_retweet_count'] = tweets_df['tweet_retweet_count'].fillna('0')
tweets_df['tweet_like_count'] = tweets_df['tweet_like_count'].fillna('0')

#convert counts into integer
tweets_df['tweet_reply_count'] = tweets_df['tweet_reply_count'].astype(int)
tweets_df['tweet_retweet_count'] = tweets_df['tweet_retweet_count'].astype(int)
tweets_df['tweet_like_count'] = tweets_df['tweet_like_count'].astype(int)


#SET the tweet content lower case for further comparison
tweets_df['tweet_content'] = tweets_df['tweet_content'].str.lower()

#Clean Timestamp of Tweets
tweets_df['tweet_date'] = tweets_df['tweet_date'].str.replace(r'\.', '') #remove dot (.)
tweets_df['tweet_date'] = tweets_df['tweet_date'].str.replace(r'000Z$', '') #remove last 4 chars
tweets_df['tweet_date'] = pd.to_datetime(tweets_df['tweet_date'], format='%Y-%m-%dT%H:%M:%S') #converting data into Datetime

#Tweets Date has been UTC timezone by default and needed to transform to ETC
tweets_df.tweet_date = tweets_df.tweet_date.dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

In [3]:
#save all Tweets

tweets_df.to_csv('data/tweets_elonmusk_all_CLEAN.csv', index=False)

In [4]:
#FILTER FOR KEY WORDS
includeKeywords = [
    'doge', 
    'bitcoin', 
    'crypto', 
    'coin', 
    'dogecoin', 
    'eth', 
    'btc', 
    'currency', 
    'cryptocurrency', 
    'dogecoin',
    'ethereum',
    'bloc00chain',
    'coinbase'
    'ripple',
    'bitcoincash',
    'xrp'
]

#.stack to make tweets_df a Series and enable us to use the pandas.Series.str accessor functions
#.str.contains with '|'.join(includeKeywords)
#.any with argument level=0 because we added a level to the index when we stacked

tweets_df = tweets_df[tweets_df.stack().str.contains('|'.join(includeKeywords)).any(level=0)]

In [5]:
#save filtered Tweets

tweets_df.to_csv('data/tweets_elonmusk_crypto_CLEAN.csv', index=False)